# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
import base64
from io import BytesIO
from dotenv import load_dotenv
import openai
from openai import OpenAI
from PIL import Image

class GptModel:
    """Wrapper around the ChatGPT calls."""
    def __init__(self, system_message=None):
        self.system_message = system_message
        
        load_dotenv(override=True)
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            raise Exception("OpenAI API Key not set")
        self.model_name = "gpt-4o-mini"
        self.api = OpenAI()

    def set_system_message(self, system_message):
        """Set this to apply a system message"""
        self.system_message = system_message
    
    def chat(self, message, history, system_message):
        messages = self.to_llm_messages(message, history)
        response = self.api.chat.completions.create(model=self.model_name, messages=messages)
        return response.choices[0].message.content

    def image(self, prompt):
        """Generate an image from the prompt"""
        image_response = openai.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
        image_base64 = image_response.data[0].b64_json
        image_data = base64.b64decode(image_base64)
        return Image.open(BytesIO(image_data))        

    def to_llm_messages(self, message, history):
        """Convert UI message and history to LLM format."""
        messages = []
        if self.system_message:
            messages += [{"role": "system", "content": AirlineAssistant.SYSTEM_PROMPT}]
        messages += history
        messages += [{"role": "user", "content": message}]
        return messages



In [ ]:
class TicketSystem:
    """Handle requests to the ticketing system."""
    UNKNOWN_PRICE = -1  # Unknown or incomplete pricing
    TICKET_PRICES = {
        "paris": 699,
        "london": 649,
        "madrid": 719,
        "rome": 709,
        "berlin": 599,
        "baltimore": 439,
        "washington dc": 349,
        "seattle": 549,
        "vancouver": 599,
        "san diego": 549,
    }
    
    def lookup_price(self, city):
        city = city.lower()
        if city in TicketSystem.TICKET_PRICES:
            return TicketSystem.TICKET_PRICES[city]
        return TicketSystem.UNKNOWN_PRICE

    def purchase(self, city):
        city = city.lower()
        if city in TicketSystem.TICKET_PRICES:
            return TicketSystem.TICKET_PRICES[city]
        
        raise Exception(f"Could not complete the ticket purchase to {city}.")


In [ ]:
import json

class AirlineAssistant:
    """Help user and perform actions with ticketing."""

    SYSTEM_PROMPT = "You are an airline ticketing assistant. You should help the user " +\
        "with air travel questions and ticket purchases. You responses should be cheerful and helpful. " +\
        "Don't be afraid to say if you don't know the answer or can't do something."

    PRICE_FUNCTION = {
        "name": "lookup_ticket_price",
        "description": "Get the price of a round trip return ticket to the destination city. Call this whenever you need to know a ticket price, for example when a customer asks 'How much is a ticket to this city'",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }

    }
    
    PURCHASE_FUNCTION = {
        "name": "purchase_ticket",
        "description": "Purchases the return ticket to the destination city. Call this whenever you need to purchase a specific ticket to a city,"
                        "for example when a customer asks 'Purchase a ticket to Paris.'",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }

    }
    
    def __init__(self, model, ticket_system):
        self.model = model
        self.ticket_system = ticket_system
        self.tools = [
            {"type": "function", "function": AirlineAssistant.PRICE_FUNCTION},
            {"type": "function", "function": AirlineAssistant.PURCHASE_FUNCTION},
        ]

    def chat(self, message, history):
        """Performs the interactions with the model with the user's message."""
        # TODO: Model might really do this part
        model_name = self.model.model_name
        messages = self.model.to_llm_messages(message, history)
        response = openai.chat.completions.create(model=model_name, messages=messages, tools=self.tools)
        # TOOD: Add streaming

        if response.choices[0].finish_reason == "tool_calls":
            # Call the requested tool
            tool_request = response.choices[0].message
            tool_response, city = self.handle_tool_call(tool_request)

            # Append the tool request/response and call model again
            messages.append(tool_request)
            messages.append(tool_response)
            response = openai.chat.completions.create(model=model_name, messages=messages)

        response_message = response.choices[0].message.content

        import base64
        from io import BytesIO
        from PIL import Image
        from IPython.display import Audio, display
        
        def talker(message):
            response = openai.audio.speech.create(
                model="tts-1",
                voice="onyx",
                input=message)
        
            audio_stream = BytesIO(response.content)
            output_filename = "output_audio.mp3"
            with open(output_filename, "wb") as f:
                f.write(audio_stream.read())
        
            # Play the generated audio
            display(Audio(output_filename, autoplay=True))
        
        talker(response_message)
        return response_message
        

    def handle_tool_call(self, message):
        tool_call = message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get("destination_city")

        # Create response content by calling ticketing
        content = {"destination_city": city, "price": TicketSystem.UNKNOWN_PRICE}
        try:
            price = TicketSystem.UNKNOWN_PRICE
            if function_name == "lookup_ticket_price":
                price = self.ticket_system.lookup_price(city)
                content["price"] = price
            elif function_name == "purchase_ticket":
                price = self.ticket_system.purchase(city)
                content["price"] = price
                if price != TicketSystem.UNKNOWN_PRICE:
                    content["status"] = "completed"
            else:
                raise Exception(f"An invalid request could not be handled: {function_name}")

            # Validate pricing
            if price == TicketSystem.UNKNOWN_PRICE:
                content["error"] = f"Pricing is unavailale for {city}"
        except Exception as exc:
            content["error"] = str(exc)

        # Create full tool response
        response = {
            "role": "tool",
            "content": json.dumps(content),
            "tool_call_id": tool_call.id,
        }

        if function_name == "purchase_ticket" and "error" not in content:
            image = self.model.image(city)
            display(image)
            
        return response, city



In [ ]:
import gradio as gr

def run_airline_agent():
    """Creates and wires all the components and runs!""" 
    model = GptModel()
    ticket_system = TicketSystem()
    assistant = AirlineAssistant(model, ticket_system)
    chat_ui = gr.ChatInterface(fn=assistant.chat, type="messages")
    chat_ui.launch()


In [ ]:
# Put it all together and run the agent!
run_airline_agent()
